In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import utils
import models1
import time

In [38]:
torch.manual_seed(1)

In [3]:
# # Data Path
TRAIN_DATA_CONTEXT_PATH = '/content/drive/MyDrive/model/data/train_context_new.txt'
TRAIN_DATA_CONTENT_PATH_1 = '/content/drive/MyDrive/model/data/train_content_1_new.txt'
TRAIN_DATA_CONTENT_PATH_2 = '/content/drive/MyDrive/model/data/train_content_2_new.txt'

# # Hyperparamters
CONTEXT_SIZE = 1
CONTENT_CONTEXT_SIZE = 2
EMBEDDING_DIM = 300
ALPHA = 0.2
BETA = 0.4

# # Training Parameters
LEARNING_RATE = 0.001
EPOCHS = 60
CONTEXT_BATCH_SIZE = 250  # 256
CONTENT_1_BATCH_SIZE = 20000
CONTENT_2_BATCH_SIZE = 5000  #8192
NUM_SAMPLED = 20

In [6]:
print("Data Loading..")
train_context, train_content_1, train_content_2 = \
utils.load_data(TRAIN_DATA_CONTEXT_PATH, TRAIN_DATA_CONTENT_PATH_1, TRAIN_DATA_CONTENT_PATH_2)
print("Data Indexing..")
poi2id, id2poi, word2id, id2word, label2id, id2label, train_context, train_content_1, train_content_2 = \
utils.indexing(train_context, train_content_1, train_content_2)
print("Dataset Processing..")
train_context, train_content_1, train_content_2 = \
utils.process_data(train_context, train_content_1, train_content_2, CONTEXT_SIZE)
print("Data Statistics:\nNumber of POIs: {}\nNumber of Words: {}\nNumber of Labels: {}".format(
    len(poi2id)-1, len(word2id)-1, len(label2id)-1))

Data Loading..


In [11]:
# Training
# # Model Initialize
model = models1.CAPE(EMBEDDING_DIM, len(poi2id), len(word2id), len(label2id)).cuda()
criterion = nn.LogSigmoid()

In [12]:
# optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)
optimizer_context = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)
optimizer_content_1 = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)
optimizer_content_2 = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)

In [21]:
print("=======================================")
print("Training..\n")

best_context_loss = 1e9
best_content_loss_1 = 1e9
best_content_loss_2 = 1e9

for i in range(EPOCHS):

  t = time.time()
  # Check-in Context Layer
  batches = utils.train_context_batches(train_context, CONTEXT_BATCH_SIZE)
  step = 0
  batch_num = int(len(train_context)/CONTEXT_BATCH_SIZE) + 1
  context_loss = 0
  for batch in batches:
    target, context = batch
    input_target = Variable(torch.cuda.LongTensor(target))
    input_context = Variable(torch.cuda.LongTensor(context))

    # Optimizer Initialize
    optimizer_context.zero_grad()
    positive, negative = model(input_target, input_context, NUM_SAMPLED)
    _loss = -(criterion(positive) + criterion(negative).mean()).sum()
    _loss.backward()
    optimizer_context.step()
    context_loss += _loss.cpu().data.numpy()

  context_loss /= CONTEXT_BATCH_SIZE
  print("At Epoch: [{}/{}] Time {}s Context Loss: {}\n".format(i+1, EPOCHS, round(time.time() - t), context_loss))

  batches = utils.train_content_batches(train_content_1, CONTENT_1_BATCH_SIZE)
  step = 0
  batch_num = int(len(train_content_1)/CONTENT_1_BATCH_SIZE) + 1
  content_loss_1 = 0
  for batch in batches:
    poi, target, context = batch
    input_poi = Variable(torch.cuda.LongTensor(poi))
    input_target = Variable(torch.cuda.LongTensor(target))
    input_context = Variable(torch.cuda.LongTensor(context))

    # Optimizer Initialize
    optimizer_content_1.zero_grad()
    positive, negative = model.content_1(input_poi, input_target, input_context, NUM_SAMPLED)
    _loss = -1*ALPHA*(criterion(positive)+criterion(negative).mean()).sum()
    _loss.backward()
    optimizer_content_1.step()
    content_loss_1 += _loss.cpu().data.numpy()

  content_loss_1 /= CONTENT_1_BATCH_SIZE
  print("At Epoch: [{}/{}] Time {}s Content Loss 1: {}\n".format(i+1, EPOCHS, round(time.time() - t), content_loss_1))

  batches = utils.train_content_batches(train_content_2, CONTENT_2_BATCH_SIZE)
  step = 0
  batch_num = int(len(train_content_2)/CONTENT_2_BATCH_SIZE) + 1
  content_loss_2 = 0
  for batch in batches:
    poi, target, context = batch
    input_poi = Variable(torch.cuda.LongTensor(poi))
    input_target = Variable(torch.cuda.LongTensor(target))
    input_context = Variable(torch.cuda.LongTensor(context))

    # Optimizer Initialize
    optimizer_content_2.zero_grad()
    positive, negative = model.content_2(input_poi, input_target, input_context, NUM_SAMPLED)
    _loss = -1*BETA*(criterion(positive)+criterion(negative).mean()).sum()
    _loss.backward()
    optimizer_content_2.step()
    content_loss_2 += _loss.cpu().data.numpy()

  content_loss_2 /= CONTENT_2_BATCH_SIZE
  print("At Epoch: [{}/{}] Time {}s Content Loss 2: {}\n".format(i+1, EPOCHS, round(time.time() - t), content_loss_2))

Training..

Process Context Layer Epoch: [1/20000] Batch: [1/137]
Process Context Layer Epoch: [1/20000] Batch: [2/137]
Process Context Layer Epoch: [1/20000] Batch: [3/137]
Process Context Layer Epoch: [1/20000] Batch: [4/137]
Process Context Layer Epoch: [1/20000] Batch: [5/137]
Process Context Layer Epoch: [1/20000] Batch: [6/137]
Process Context Layer Epoch: [1/20000] Batch: [7/137]
Process Context Layer Epoch: [1/20000] Batch: [8/137]
Process Context Layer Epoch: [1/20000] Batch: [9/137]
Process Context Layer Epoch: [1/20000] Batch: [10/137]
Process Context Layer Epoch: [1/20000] Batch: [11/137]
Process Context Layer Epoch: [1/20000] Batch: [12/137]
Process Context Layer Epoch: [1/20000] Batch: [13/137]
Process Context Layer Epoch: [1/20000] Batch: [14/137]
Process Context Layer Epoch: [1/20000] Batch: [15/137]
Process Context Layer Epoch: [1/20000] Batch: [16/137]
Process Context Layer Epoch: [1/20000] Batch: [17/137]
Process Context Layer Epoch: [1/20000] Batch: [18/137]
Process

Process Context Layer Epoch: [1/20000] Batch: [136/137]
Process Context Layer Epoch: [1/20000] Batch: [137/137]
Process Content Layer Epoch: [1/20000] Batch: [1/16141]
Process Content Layer Epoch: [1/20000] Batch: [2/16141]
Process Content Layer Epoch: [1/20000] Batch: [3/16141]
Process Content Layer Epoch: [1/20000] Batch: [4/16141]
Process Content Layer Epoch: [1/20000] Batch: [5/16141]
Process Content Layer Epoch: [1/20000] Batch: [6/16141]
Process Content Layer Epoch: [1/20000] Batch: [7/16141]
Process Content Layer Epoch: [1/20000] Batch: [8/16141]
Process Content Layer Epoch: [1/20000] Batch: [9/16141]
Process Content Layer Epoch: [1/20000] Batch: [10/16141]
Process Content Layer Epoch: [1/20000] Batch: [11/16141]
Process Content Layer Epoch: [1/20000] Batch: [12/16141]
Process Content Layer Epoch: [1/20000] Batch: [13/16141]
Process Content Layer Epoch: [1/20000] Batch: [14/16141]
Process Content Layer Epoch: [1/20000] Batch: [15/16141]
Process Content Layer Epoch: [1/20000] Bat

Process Content Layer Epoch: [1/20000] Batch: [129/16141]
Process Content Layer Epoch: [1/20000] Batch: [130/16141]
Process Content Layer Epoch: [1/20000] Batch: [131/16141]
Process Content Layer Epoch: [1/20000] Batch: [132/16141]
Process Content Layer Epoch: [1/20000] Batch: [133/16141]
Process Content Layer Epoch: [1/20000] Batch: [134/16141]
Process Content Layer Epoch: [1/20000] Batch: [135/16141]
Process Content Layer Epoch: [1/20000] Batch: [136/16141]
Process Content Layer Epoch: [1/20000] Batch: [137/16141]
Process Content Layer Epoch: [1/20000] Batch: [138/16141]
Process Content Layer Epoch: [1/20000] Batch: [139/16141]
Process Content Layer Epoch: [1/20000] Batch: [140/16141]
Process Content Layer Epoch: [1/20000] Batch: [141/16141]
Process Content Layer Epoch: [1/20000] Batch: [142/16141]
Process Content Layer Epoch: [1/20000] Batch: [143/16141]
Process Content Layer Epoch: [1/20000] Batch: [144/16141]
Process Content Layer Epoch: [1/20000] Batch: [145/16141]
Process Conten

Process Content Layer Epoch: [1/20000] Batch: [258/16141]
Process Content Layer Epoch: [1/20000] Batch: [259/16141]
Process Content Layer Epoch: [1/20000] Batch: [260/16141]
Process Content Layer Epoch: [1/20000] Batch: [261/16141]
Process Content Layer Epoch: [1/20000] Batch: [262/16141]
Process Content Layer Epoch: [1/20000] Batch: [263/16141]
Process Content Layer Epoch: [1/20000] Batch: [264/16141]
Process Content Layer Epoch: [1/20000] Batch: [265/16141]
Process Content Layer Epoch: [1/20000] Batch: [266/16141]
Process Content Layer Epoch: [1/20000] Batch: [267/16141]
Process Content Layer Epoch: [1/20000] Batch: [268/16141]
Process Content Layer Epoch: [1/20000] Batch: [269/16141]
Process Content Layer Epoch: [1/20000] Batch: [270/16141]
Process Content Layer Epoch: [1/20000] Batch: [271/16141]
Process Content Layer Epoch: [1/20000] Batch: [272/16141]
Process Content Layer Epoch: [1/20000] Batch: [273/16141]
Process Content Layer Epoch: [1/20000] Batch: [274/16141]
Process Conten

Process Content Layer Epoch: [1/20000] Batch: [387/16141]
Process Content Layer Epoch: [1/20000] Batch: [388/16141]
Process Content Layer Epoch: [1/20000] Batch: [389/16141]
Process Content Layer Epoch: [1/20000] Batch: [390/16141]
Process Content Layer Epoch: [1/20000] Batch: [391/16141]
Process Content Layer Epoch: [1/20000] Batch: [392/16141]
Process Content Layer Epoch: [1/20000] Batch: [393/16141]
Process Content Layer Epoch: [1/20000] Batch: [394/16141]
Process Content Layer Epoch: [1/20000] Batch: [395/16141]
Process Content Layer Epoch: [1/20000] Batch: [396/16141]
Process Content Layer Epoch: [1/20000] Batch: [397/16141]
Process Content Layer Epoch: [1/20000] Batch: [398/16141]
Process Content Layer Epoch: [1/20000] Batch: [399/16141]
Process Content Layer Epoch: [1/20000] Batch: [400/16141]
Process Content Layer Epoch: [1/20000] Batch: [401/16141]
Process Content Layer Epoch: [1/20000] Batch: [402/16141]
Process Content Layer Epoch: [1/20000] Batch: [403/16141]
Process Conten

...

Process Content Layer Epoch: [1/20000] Batch: [1793/16141]
Process Content Layer Epoch: [1/20000] Batch: [1794/16141]
Process Content Layer Epoch: [1/20000] Batch: [1795/16141]
Process Content Layer Epoch: [1/20000] Batch: [1796/16141]
Process Content Layer Epoch: [1/20000] Batch: [1797/16141]
Process Content Layer Epoch: [1/20000] Batch: [1798/16141]
Process Content Layer Epoch: [1/20000] Batch: [1799/16141]
Process Content Layer Epoch: [1/20000] Batch: [1800/16141]
Process Content Layer Epoch: [1/20000] Batch: [1801/16141]
Process Content Layer Epoch: [1/20000] Batch: [1802/16141]
Process Content Layer Epoch: [1/20000] Batch: [1803/16141]
Process Content Layer Epoch: [1/20000] Batch: [1804/16141]
Process Content Layer Epoch: [1/20000] Batch: [1805/16141]
Process Content Layer Epoch: [1/20000] Batch: [1806/16141]
Process Content Layer Epoch: [1/20000] Batch: [1807/16141]
Process Content Layer Epoch: [1/20000] Batch: [1808/16141]
Process Content Layer Epoch: [1/20000] Batch: [1809/1614

Process Content Layer Epoch: [1/20000] Batch: [1920/16141]
Process Content Layer Epoch: [1/20000] Batch: [1921/16141]
Process Content Layer Epoch: [1/20000] Batch: [1922/16141]
Process Content Layer Epoch: [1/20000] Batch: [1923/16141]
Process Content Layer Epoch: [1/20000] Batch: [1924/16141]
Process Content Layer Epoch: [1/20000] Batch: [1925/16141]
Process Content Layer Epoch: [1/20000] Batch: [1926/16141]
Process Content Layer Epoch: [1/20000] Batch: [1927/16141]
Process Content Layer Epoch: [1/20000] Batch: [1928/16141]
Process Content Layer Epoch: [1/20000] Batch: [1929/16141]
Process Content Layer Epoch: [1/20000] Batch: [1930/16141]
Process Content Layer Epoch: [1/20000] Batch: [1931/16141]
Process Content Layer Epoch: [1/20000] Batch: [1932/16141]
Process Content Layer Epoch: [1/20000] Batch: [1933/16141]
Process Content Layer Epoch: [1/20000] Batch: [1934/16141]
Process Content Layer Epoch: [1/20000] Batch: [1935/16141]
Process Content Layer Epoch: [1/20000] Batch: [1936/1614

Process Content Layer Epoch: [1/20000] Batch: [2047/16141]
Process Content Layer Epoch: [1/20000] Batch: [2048/16141]
Process Content Layer Epoch: [1/20000] Batch: [2049/16141]
Process Content Layer Epoch: [1/20000] Batch: [2050/16141]
Process Content Layer Epoch: [1/20000] Batch: [2051/16141]
Process Content Layer Epoch: [1/20000] Batch: [2052/16141]
Process Content Layer Epoch: [1/20000] Batch: [2053/16141]
Process Content Layer Epoch: [1/20000] Batch: [2054/16141]
Process Content Layer Epoch: [1/20000] Batch: [2055/16141]
Process Content Layer Epoch: [1/20000] Batch: [2056/16141]
Process Content Layer Epoch: [1/20000] Batch: [2057/16141]
Process Content Layer Epoch: [1/20000] Batch: [2058/16141]
Process Content Layer Epoch: [1/20000] Batch: [2059/16141]
Process Content Layer Epoch: [1/20000] Batch: [2060/16141]
Process Content Layer Epoch: [1/20000] Batch: [2061/16141]
Process Content Layer Epoch: [1/20000] Batch: [2062/16141]
Process Content Layer Epoch: [1/20000] Batch: [2063/1614

Process Content Layer Epoch: [1/20000] Batch: [2174/16141]
Process Content Layer Epoch: [1/20000] Batch: [2175/16141]
Process Content Layer Epoch: [1/20000] Batch: [2176/16141]
Process Content Layer Epoch: [1/20000] Batch: [2177/16141]
Process Content Layer Epoch: [1/20000] Batch: [2178/16141]
Process Content Layer Epoch: [1/20000] Batch: [2179/16141]
Process Content Layer Epoch: [1/20000] Batch: [2180/16141]
Process Content Layer Epoch: [1/20000] Batch: [2181/16141]
Process Content Layer Epoch: [1/20000] Batch: [2182/16141]
Process Content Layer Epoch: [1/20000] Batch: [2183/16141]
Process Content Layer Epoch: [1/20000] Batch: [2184/16141]
Process Content Layer Epoch: [1/20000] Batch: [2185/16141]
Process Content Layer Epoch: [1/20000] Batch: [2186/16141]
Process Content Layer Epoch: [1/20000] Batch: [2187/16141]
Process Content Layer Epoch: [1/20000] Batch: [2188/16141]
Process Content Layer Epoch: [1/20000] Batch: [2189/16141]
Process Content Layer Epoch: [1/20000] Batch: [2190/1614

KeyboardInterrupt: 

In [22]:
model.get_embedding()
weight = model.poi_embedding.weight.cpu().data.numpy()
name = 'a_0.2_b_0.4_new'
np.savetxt('/content/drive/MyDrive/model/data/{}.txt'.format(name), weight)